# **Documentación de Archivos PHP**

# **1. db.php**

Este archivo se encarga de establecer una conexión a la base de datos utilizando PDO `(PHP Data Objects)`.

# **Funcionalidad**

**Conexión a la base de datos**:

Define las variables necesarias para la conexión, como el host, nombre de la base de datos, usuario y contraseña.
Manejo de excepciones: Utiliza un bloque try-catch para manejar errores en la conexión. Si ocurre un error, se muestra un mensaje y se detiene la ejecución.

In [ ]:
$host = 'localhost'; // Define el host de la base de datos
$dbname = 'mi_base_datos'; // Define el nombre de la base de datos
$user = 'root'; // Define el nombre de usuario para la conexión
$password = ''; // Define la contraseña para la conexión

try {
    // Intenta crear una nueva conexión PDO a la base de datos
    $conn = new PDO("mysql:host=$host;dbname=$dbname;charset=utf8", $user, $password);
    // Configura el modo de error de PDO para lanzar excepciones
    $conn->setAttribute(PDO::ATTR_ERRMODE, PDO::ERRMODE_EXCEPTION);
} catch (PDOException $e) {
    // Si hay un error en la conexión, se captura y se muestra un mensaje
    die("Error al conectar con la base de datos: " . $e->getMessage());
}

**Arquitectura**

Se utiliza la clase `PDO` para establecer la conexión.

Se configuran atributos para manejar excepciones en caso de errores.

# **2. login.php**

Este archivo maneja el proceso de inicio de sesión de un usuario.

# **Funcionalidad**

**Verificación del método de solicitud**: Comprueba si la solicitud es de tipo POST, lo que indica que se ha enviado un formulario de inicio de sesión.
Consulta de usuario: Prepara y ejecuta una consulta para buscar el usuario en la base de datos.

**Verificación de credenciales**: Comprueba si el usuario existe y si la contraseña proporcionada es correcta utilizando `password_verify().`

In [ ]:
require 'db.php'; // Incluye el archivo de conexión a la base de datos

if ($_SERVER['REQUEST_METHOD'] === 'POST') { // Verifica si la solicitud es de tipo POST
    $username = $_POST['username']; // Obtiene el nombre de usuario del formulario
    $password = $_POST['password']; // Obtiene la contraseña del formulario

    // Prepara una consulta para buscar el usuario en la base de datos
    $stmt = $conn->prepare("SELECT * FROM users WHERE username = :username");
    $stmt->bindParam(':username', $username); // Vincula el parámetro :username con el valor de $username
    $stmt->execute(); // Ejecuta la consulta

    // Obtiene los datos del usuario como un arreglo asociativo
    $user = $stmt->fetch(PDO::FETCH_ASSOC);

    // Verifica si el usuario existe y si la contraseña proporcionada es correcta
    if ($user && password_verify($password, $user['password'])) {
        // Si las credenciales son válidas, muestra un mensaje de bienvenida
        echo "Bienvenido, " . htmlspecialchars($username) . "!";
    } else {
        // Si las credenciales son incorrectas, muestra un mensaje de error
        echo "Usuario o contraseña incorrectos.";
    }
}

**Arquitectura**

Utiliza declaraciones preparadas para evitar inyecciones SQL.

Se utiliza `htmlspecialchars()` para prevenir ataques XSS al mostrar el nombre de usuario.

# **3. register.php**

Este archivo permite el registro de nuevos usuarios en la base de datos.

# **Funcionalidad**

**Verificación del método de solicitud**: Comprueba si la solicitud es de tipo POST para procesar el formulario de registro.

**Comprobación de existencia de usuario**: Verifica si el nombre de usuario ya está registrado.

**Registro de nuevo usuario**: Si el usuario no existe, se inserta en la base de datos con la contraseña cifrada.

In [ ]:
require 'db.php'; // Incluye el archivo de conexión a la base de datos

if ($_SERVER['REQUEST_METHOD'] === 'POST') { // Verifica si la solicitud es de tipo POST
    $username = $_POST['username']; // Obtiene el nombre de usuario del formulario
    $password = $_POST['password']; // Obtiene la contraseña del formulario

    // Prepara una consulta para verificar si el usuario ya existe
    $stmt = $conn->prepare("SELECT * FROM users WHERE username = :username");
    $stmt->bindParam(':username', $username); // Vincula el parámetro :username con el valor de $username
    $stmt->execute(); // Ejecuta la consulta

    if ($stmt->rowCount() > 0) { // Verifica si hay resultados (usuario ya existe)
        echo "El usuario ya está registrado."; // Mensaje si el usuario ya existe
    } else {
        // Cifra la contraseña antes de almacenarla
        $hashed_password = password_hash($password, PASSWORD_DEFAULT);

        // Prepara una consulta para insertar el nuevo usuario
        $stmt = $conn->prepare("INSERT INTO users (username, password) VALUES (:username, :password)");
        $stmt->bindParam(':username', $username); // Vincula el parámetro :username
        $stmt->bindParam(':password', $hashed_password); // Vincula el parámetro :password

        if ($stmt->execute()) { // Ejecuta la consulta de inserción
            echo "Usuario registrado exitosamente."; // Mensaje de éxito
        } else {
            echo "Error al registrar el usuario."; // Mensaje de error
        }
    }
}

**Arquitectura**

Utiliza `password_hash()` para cifrar la contraseña del usuario antes de almacenarla en la base de datos.

Se utilizan declaraciones preparadas para evitar inyecciones SQL.


# **4. test.php**

Este archivo se utiliza para verificar la conexión a la base de datos.

# **Funcionalidad**

**Verificación de conexión**: Comprueba si la conexión a la base de datos se ha establecido correctamente y muestra un mensaje correspondiente.

In [ ]:
require 'db.php'; // Incluye el archivo de conexión a la base de datos

if ($conn) { // Verifica si la conexión fue exitosa
    echo "Conexión exitosa a la base de datos."; // Mensaje de éxito
} else {
    echo "Error al conectar."; // Mensaje de error
}

**Arquitectura**

Simplemente verifica si la variable `$conn` está establecida, lo que indica que la conexión fue exitosa.
